In [1]:
from runner import evaluate_performance
stats = evaluate_performance()
print("Decision Tree Accuracy_Mariam = ", stats[0, 0], " (", stats[0, 1], ")")
print("Random Forest Tree Accuracy_Hayk = ", stats[1, 0], " (", stats[1, 1], ")")
print("Logistic Reg. Accuracy_David = ", stats[2, 0], " (", stats[2, 1], ")")

Decision Tree Accuracy_Mariam =  0.746153846154  ( 0.0624926031126 )
Random Forest Tree Accuracy_Hayk =  0.769230769231  ( 0.0516015687115 )
Logistic Reg. Accuracy_David =  0.823076923077  ( 0.0670599837468 )
